In [6]:
import csv
from math import log
import numpy as np
from array import array
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error
import math

policy_ID = 'PolicyNo' # name of the first column for identifying the policies

sampling_data = '../../data/cleaned_training_data_2016_ver_02.csv'

train_data = ''
test_data = ''

## Here features and targets are defined

In [10]:
features = ['Driver_Total', 'Driver_Total_Male', 'Driver_Total_Female', 'Driver_Total_Single', 'Driver_Total_Married','Driver_Minimum_Age', 'Driver_Maximum_Age', 'Driver_Total_Teenager_Age_15_19', 'Driver_Total_College_Ages_20_23', 'Driver_Total_Young_Adult_Ages_24_29', 'Annual_Premium']#, 'Driver_Total_Low_Middle_Adult_Ages_30_39', 'Driver_Total_Middle_Adult_Ages_40_49', 'Driver_Total_Adult_Ages_50_64', 'Driver_Total_Senior_Ages_65_69', 'Driver_Total_Upper_Senior_Ages_70_plus']
annual_prem_ftr = ['Annual_Premium']
all_target = ['Claim_Count', 'Loss_Amount', 'Frequency', 'Severity', 'Loss_Ratio']
#target_2 = ['Loss_Ratio']
target_2 = ['Claim_Count']

df = pd.read_csv(sampling_data)
df = df.drop(policy_ID, 1)
#data_sample = df

## This part takes out outliers based on Loss_Amount

In [11]:
#df = df[df['Loss_Amount'] < 10000]

#df = df[df['Loss_Ratio'] == 0]
#df.shape
#complete_data_frame = df
#df.shape

In [14]:
#df = df.take(np.random.permutation(len(df))[:1000])
df.shape

(424422, 68)

In [18]:
train_data,test_data = train_test_split(df, test_size = 0.2)

## 0. Regression

In [19]:
X_test = test_data[features]
print X_test.columns.values
y_test = test_data[target_2]
X = train_data[features].values
y = train_data[target_2].values

['Driver_Total' 'Driver_Total_Male' 'Driver_Total_Female'
 'Driver_Total_Single' 'Driver_Total_Married' 'Driver_Minimum_Age'
 'Driver_Maximum_Age' 'Driver_Total_Teenager_Age_15_19'
 'Driver_Total_College_Ages_20_23' 'Driver_Total_Young_Adult_Ages_24_29'
 'Annual_Premium']


### 0.1. Ordinary Least Squares

In [7]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X,y)
rmse = math.sqrt(mean_squared_error(y_test, regr.predict(X_test))) # y_test -> true, X_test <- predicted
print('RMSE: %.2f' % rmse)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))
regr.coef_

RMSE: 0.20
Variance score: 0.09


array([[-0.01284392, -0.0086026 , -0.00424132, -0.05215819, -0.03801597,
        -0.00293223,  0.00215154, -0.05000768, -0.02635404, -0.01584545,
         0.00019298]])

### 0.2 Ridge Regression

In [8]:
from sklearn import linear_model
regr = linear_model.Ridge (alpha = .5)
regr.fit(X, y) 

rmse = math.sqrt(mean_squared_error(y_test, regr.predict(X_test))) # y_test -> true, X_test <- predicted
print('RMSE: %.2f' % rmse)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))

print regr.coef_

print regr.intercept_ 


RMSE: 0.20
Variance score: 0.09
[[-0.01297316 -0.00869651 -0.00427665 -0.05183211 -0.03769617 -0.00290703
   0.00213815 -0.04852688 -0.02569353 -0.01547767  0.00019269]]
[ 0.08989663]


### 0.3 Lasso

In [9]:
from sklearn import linear_model
reg = linear_model.Lasso(alpha = 0.1)
reg.fit(X, y)

rmse = math.sqrt(mean_squared_error(y_test, regr.predict(X_test))) # y_test -> true, X_test <- predicted
print('RMSE: %.2f' % rmse)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))

RMSE: 0.20
Variance score: 0.09


### 0.4. Bayesian Regression

In [10]:
from sklearn import linear_model
reg = linear_model.BayesianRidge()
y = train_data[target_2].values
reg.fit(X,y)
rmse = math.sqrt(mean_squared_error(y_test, regr.predict(X_test))) # y_test -> true, X_test <- predicted
print('RMSE: %.2f' % rmse)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))
regr.coef_

RMSE: 0.20
Variance score: 0.09


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[-0.01297316, -0.00869651, -0.00427665, -0.05183211, -0.03769617,
        -0.00290703,  0.00213815, -0.04852688, -0.02569353, -0.01547767,
         0.00019269]])

## 1. Combined Models
### 1.1. Gradient Boosting Regression

In [11]:
all_features = list(set(list(df.columns.values)) - set(all_target))
X_test = test_data[features]
y_test = test_data[target_2]
X = train_data[features].values
y = train_data[target_2].values

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_friedman1

In [ ]:
params = {'n_estimators': 100, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = GradientBoostingRegressor(**params)

clf.fit(X, y.ravel())
rmse = math.sqrt(mean_squared_error(y_test, clf.predict(X_test))) # y_test -> true, X_test <- predicted
print('RMSE: %.2f' % rmse)
#clf.train_score_

### 1.2. 

In [ ]:
X = train_data[all_features]
X = X.select_dtypes(exclude=['object'])

clf = GradientBoostingRegressor(n_estimators=10, learning_rate=1.0).fit(X, y)
clf.feature_importances_

In [ ]:
###############################################################################
# Plot feature importance 
feature_importance = clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
print type(feature_importance)
feature_importance = feature_importance[feature_importance>10]
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, X.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()


# Prediction with important features as identified by gbr algorithm

In [ ]:
features = list(X.columns[sorted_idx])
print features
X_test = test_data[features].values
y_test = test_data[target_1].values
X_train = train_data[features].values
y_train = train_data[target_1].values

In [ ]:
params = {'n_estimators': 200, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)
rmse = math.sqrt(mean_squared_error(y_test, clf.predict(X_test))) # y_test -> true, X_test <- predicted
print('RMSE: %.2f' % rmse)
clf.train_score_.mean()
clf.score(X_test, y_test)
#from sklearn import metrics
#metrics.accuracy_score(y_train, Predict(X_train))

In [ ]:
features = list(X.columns[sorted_idx])
print features
X_test = test_data[features].values
y_test = test_data[target_1].values
X_train = train_data[features].values
y_train = train_data[target_1].values
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X_train,y_train)
rmse = math.sqrt(mean_squared_error(y_test, regr.predict(X_test))) # y_test -> true, X_test <- predicted
print('RMSE: %.2f' % rmse)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))
regr.coef_